In [1]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import torch
from MTGNN import MTGNN
from utils import load_model
import pickle

%matplotlib inline

In [2]:
data_path = "./y_outputs.pickle"
data = pd.read_pickle(data_path)

In [14]:
data

,y_pred,y_true
0,"[[[[54.713753], [86.04455], [103.88353], [111....","[[[[0.], [-0.3], [-0.3], [-0.3], [-0.3], [-0.3..."


In [15]:
data["y_pred"][0].shape

(134, 1, 288, 1)

In [16]:
data["y_true"].shape

(1,)

In [24]:
y_pred_data = data['y_pred'].apply(np.squeeze)

In [25]:
y_pred_data[0].shape

(134, 288)

In [26]:
y_true_data = data['y_true'].apply(np.squeeze)

In [27]:
y_true_data[0].shape

(134, 288)

In [28]:
y_pred_data = np.stack(y_pred_data).squeeze()
y_true_data = np.stack(y_true_data).squeeze()

In [29]:
y_pred_data.shape

(134, 288)

In [11]:
y_df = pd.DataFrame({"Day":np.repeat(np.tile(np.arange(11), 69), 288),
            "TurbID": np.repeat(np.arange(69), 11 * 288),
            "Horizon": np.tile(np.arange(288), 11 * 69),
            "y_pred": y_pred_data.flatten(),
            "y_true": y_true_data.flatten()})
y_df.head()

,Day,TurbID,Horizon,y_pred,y_true
0,0,0,0,6838715.0,9233202.0
1,0,0,1,6853226.5,9612715.0
2,0,0,2,7042043.0,10061649.0
3,0,0,3,6932486.5,10363480.0
4,0,0,4,6888359.5,10515326.0


In [31]:
y_df["time_idx"] = np.arange(len(np.repeat(np.tile(np.arange(11), 69), 288)))

In [34]:
time_index = np.arange(11 * 288)

In [35]:
np.tile(time_index, 69)

array([   0,    1,    2, ..., 3165, 3166, 3167])

In [36]:
y_df["time_index"] = np.tile(time_index, 69)

In [38]:
y_df[y_df["time_index"] == 0]

,Day,TurbID,Horizon,y_pred,y_true,time_idx,time_index
0,0,0,0,6838715.0,9233202.0,0,0
3168,0,1,0,6874643.0,9620938.0,3168,0
6336,0,2,0,4959850.5,7736914.0,6336,0
9504,0,3,0,1995774.0,2973379.0,9504,0
12672,0,4,0,5008997.5,7574201.0,12672,0
...,...,...,...,...,...,...,...
202752,0,64,0,8288286.0,10784332.0,202752,0
205920,0,65,0,8098092.0,10761050.0,205920,0
209088,0,66,0,6198503.5,9448084.0,209088,0
212256,0,67,0,6997430.5,9177730.0,212256,0


In [30]:
import hvplot
import hvplot.pandas
import hvplot.xarray
import holoviews as hv

In [27]:
y_xarr = y_df.set_index(["TurbID", "Day", "time_idx"]).to_xarray()

In [28]:
y_xarr

<xarray.Dataset>
Dimensions:   (TurbID: 69, Day: 11, time_idx: 218592)
Coordinates:
  * TurbID    (TurbID) int64 0 1 2 3 4 5 6 7 8 9 ... 60 61 62 63 64 65 66 67 68
  * Day       (Day) int64 0 1 2 3 4 5 6 7 8 9 10
  * time_idx  (time_idx) int64 0 1 2 3 4 ... 218587 218588 218589 218590 218591
Data variables:
    Horizon   (TurbID, Day, time_idx) float64 0.0 1.0 2.0 ... 285.0 286.0 287.0
    y_pred    (TurbID, Day, time_idx) float32 6.839e+06 6.853e+06 ... 4.76e+06
    y_true    (TurbID, Day, time_idx) float32 9.233e+06 9.613e+06 ... 1.153e+07

In [30]:
y_xarr.isel(Day=slice(0, 1)).sel(TurbID=[0, 1, 2, 3]).hvplot.line(x="time_idx", y="y_pred", by="TurbID")

BokehModel(combine_events=True, render_bundle={'docs_json': {'d6fb0cd9-34c7-4cf2-963a-62347baadb43': {'version…

In [58]:
px.line(y_df[:30000], x="time_idx", y=["y_pred", "y_true"], line_group="TurbID")

In [31]:
time_values = pd.date_range(start='2012-01-01', periods=288, freq='5T')

In [43]:
import xarray as xr
power_data = xr.DataArray(y_pred_data, dims=('day', 'turbinID', 'time'), coords={'day': np.arange(0, 1), 'turbinID': np.arange(1,70), 'time':time_values})

In [34]:
import xarray as xr
pwr_dat = xr.Dataset(
    data_vars=dict(
        y_pred=(['turbineID', 'time'], y_pred_data),
        y_true=(['turbineID', 'time'], y_true_data)
    ),
    coords=dict(
        turbineID= np.arange(0,134),
        time=time_values
    )
)
pwr_dat

<xarray.Dataset>
Dimensions:    (turbineID: 134, time: 288)
Coordinates:
  * turbineID  (turbineID) int64 0 1 2 3 4 5 6 7 ... 127 128 129 130 131 132 133
  * time       (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:55:00
Data variables:
    y_pred     (turbineID, time) float32 54.71 86.04 103.9 ... 437.9 451.1 443.5
    y_true     (turbineID, time) float32 0.0 -0.3 -0.3 ... 137.2 195.2 291.6

In [35]:
power_data = xr.Dataset({'y_pred': power_data})

NameError: name 'power_data' is not defined

In [36]:
power_data

NameError: name 'power_data' is not defined

In [50]:
power_data.isel(time=slice(0,10)).hvplot.line(x='time', y='y_pred', by='turbinID')

BokehModel(combine_events=True, render_bundle={'docs_json': {'bda106cb-b3a6-45b4-8a14-b8ea3cd90b95': {'version…

In [38]:
pwr_dat.sel(turbineID=[0,1,2]).hvplot.line(x='time', y=['y_pred','y_true'])

BokehModel(combine_events=True, render_bundle={'docs_json': {'7d349d11-8047-4e69-ba9e-e5098f9f1eb1': {'version…